In [1]:
import os
import json
import warnings
from embedchain import App
import whisper
warnings.filterwarnings('ignore')

from nlp import Hear2Learn

In [2]:
h2l = Hear2Learn()

In [3]:
with open('example_transcript.txt') as f:
    text = f.read()
summaries = h2l.summarize_text(text)

In [6]:
quiz = h2l.generate_quiz(str(summaries))

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  4.06it/s]


In [ ]:
quiz

In [ ]:
with open('example_transcript.txt') as f:
    text = f.read()
summary = summarize_text(text)
for point in summary.split('\n'):
    print(point)

In [ ]:
d = {}
for s in summary.split('Topic: ')[1:]:
    topic_summs = s.split('\n')
    topic = topic_summs[0]
    points = topic_summs[1:]
    d[topic] = [point.strip('-').strip() for point in points if len(point) > 1]

In [ ]:
with open('example_transcript.txt') as f:
    text = f.read()
with open('../creds.json') as f:
        creds = json.load(f)
        
os.environ["HUGGINGFACE_ACCESS_TOKEN"] = creds['HFTOKEN'] #put your access token here

app = App.from_config(config={
    "llm": {
        "provider": "huggingface",
        "config": {
            "model": 'mistralai/Mixtral-8x7B-Instruct-v0.1',
            'top_p': 0.95,
            'temperature': 0.,
        }
    },
    "embedder": {
        "provider": "huggingface",
        "config": {
            "model": 'sentence-transformers/all-mpnet-base-v2'
        }
    }
})

instruction = """
    Task: Analyze the input text and identify different topics being discussed. For each topic generate a summary. Generate the summaries as bullet points for each topic. 
    Generate atleast 5 bullet points and atmax 20 bullet points for each topic summary. 
    Provide the result in the following format: Topic\n - bullet point1\n -bullet point2\n"""
prompt = f'{instruction}\n + Text: {text}'
answer= app.chat(prompt)
summaries = answer.split('Answer')[-1]
app.delete_session_chat_history()

In [ ]:
summaries = summaries.strip(':\n')
d = {}
for s in summaries.split('Topic: ')[1:]:
    topic_summs = s.split('\n')
    topic = topic_summs[0]
    points = topic_summs[1:]
    d[topic] = [point.strip('-').strip() for point in points if len(point) > 1]

In [ ]:
app.add(str(d))

In [ ]:
quiz_instruction = """Generate a quiz with 10 to 15 questions on the topics provided to you. Generate the response in json format"""

In [ ]:
quiz = app.chat(quiz_instruction)

In [ ]:
quiz = quiz.split('Answer')[-1]

In [ ]:
for question in quiz.split('\n'):
    print(question)